<a href="https://colab.research.google.com/github/SedoyChloric/work_in_collab/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_DBSCAN_Clusterisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread import Cell
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
columns = ['a', 'b', 'c', 'd', 'e','f','g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'aa',
           'ab', 'ac', 'ad']

In [ ]:
def get_data_from_googlesheet(google_spreedsheet):
  aerogels_properties = np.array(google_spreedsheet.sheet1.get_all_values())
  name_of_properties = aerogels_properties[0][2:-1]
  aerogels_properties = np.transpose(aerogels_properties)
  id = aerogels_properties[1][1:]
  source = aerogels_properties[-1][1:]
  prop = aerogels_properties[2:-1][0:]
  return id, source, np.delete(prop, 0, 1).astype('float64'), name_of_properties

def sort_data(id, properties, source):
  table = np.array([id, properties, source])
  table = np.transpose(table)
  table = table[np.argsort(table[:, 1].astype(float))]
  table = np.transpose(table)
  return table[0], table[1].astype(float), table[2]

def find_eucledian_distance(i, j):
  return np.abs(i-j)

def add_neighbors(data, point_index, eps):
  neighbors = np.array([])
  for i in range(len(data)):
    if find_eucledian_distance(data[i], data[point_index]) <= eps:
      neighbors = np.append(neighbors, i)
  return neighbors.astype('int32')

def release_dbscan(data, eps, min_points):
  cluster_id = 0
  labels = np.zeros(len(data), dtype='int32')
  for point in range(len(data)):
    if labels[point] != 0:
      continue
    neighbors = add_neighbors(data, point, eps)
    if len(neighbors) < min_points:
      labels[point] = -1
      continue
    labels[point] = cluster_id + 1
    seed = neighbors
    while len(seed) > 0:
      current_point = seed[0]
      seed = np.delete(seed, 0)
      if labels[current_point] == -1:
        labels[current_point] = cluster_id +1
      if labels[current_point] != 0:
        continue

      labels[current_point] = cluster_id + 1
      new_neighbors = add_neighbors(data, current_point, eps)
      if len(new_neighbors) >= min_points:
        seed = np.append(seed, new_neighbors)
    cluster_id += 1
  return labels

def print_clusters(data, id, labels, minimum_points):
  noise_id = 0
  cluster_id = 0
  for i in range(max(labels)):
    cluster = data[labels == i+1]
    ids = id[labels == i+1]
    if len(cluster) <= minimum_points:
      print("Шум №", noise_id+1, "\nЗначения: \n", cluster, "\nНаименования: \n", ids)
      noise_id += 1
    else:
      print("Кластер №", cluster_id+1, "\nЗначения: \n", cluster, "\nНаименования: \n", ids)
      cluster_id +=1
  print("Количество найденных кластеров:", cluster_id,
      "\nКоличество Шума:               ", noise_id)
  return

def pack_clusters(data, labels, minimum_points):
  packed_clusters = labels.astype(str)
  noise_id = 0
  cluster_id = 0
  for i in range(max(labels)):
    cluster = data[labels == i+1]
    if len(cluster) <= minimum_points:
      packed_clusters[labels == i+1] = f"Шум №{noise_id+1}"
      noise_id += 1
    else:
      packed_clusters[labels == i+1] = f"Кластер №{cluster_id+1}"
      cluster_id +=1
  return packed_clusters

def to_list(array):
  return array.reshape(-1, 1).tolist()

def prepare_googlesheet(id_array, source_array, property_values, name_of_property, worksheet):
  worksheet.update(to_list(np.arange(len(id_array)+1)), 'a1:a')
  worksheet.update(to_list(id_array), 'b2:b')
  worksheet.update(to_list(property_values), 'c2:c')
  worksheet.update_acell('b1', name_of_property)
  worksheet.update_acell('c1', "Значение")
  worksheet.update_acell('d1', "Кластеризация")
  return

def import_result_in_googlesheet(packed_clusters, worksheet, radius, columns=columns):
  column = worksheet.find('Кластеризация').col
  worksheet.update_acell(f'{columns[column-1]}1', f'R={radius}')
  worksheet.update_acell(f'{columns[column]}1', 'Кластеризация')
  worksheet.update(to_list(packed_clusters), f'{columns[column-1]}2:{columns[column-1]}')
  return
def trim_zeros(array):
  return array[array != 0]

In [ ]:
google_spreedsheet = gc.open_by_key('13p_ZMzQnL2zga8GFjtL6A6pA40q7vTsb2g-67SYHuWY')
id_array, source_array, properties_values, name_of_properties = get_data_from_googlesheet(google_spreedsheet)
print("Есть на просторах интернета одна гугл-таблица, в которой собрано немного про характеристики аэрогелей.\n"
  "В этой прекрасной гугл-таблице насчитываются характеристики уже", len(id_array), "аэрогелей\n"
  "Эта программа может разбивать аэрогели на группы при помощи магии, которая зовётся DBScan\n"
  "Магия работает по таким характеристикам, как", name_of_properties)
while True:
  for i in range(len(name_of_properties)):
    print(i+1, "Посмотреть, что за данные храняться в", name_of_properties[i])
  print(len(name_of_properties)+1, "Начать КЛАСТЕРИЗАЦИЮ!!!!!\n",
        len(name_of_properties)+2, "Сделать вид что 'подомам'")
  try:
    choice = int(input("Введите опцию: "))
    if choice <= len(name_of_properties) and choice >= 1:
      print("Значения ", name_of_properties[choice-1], "равны:\n", properties_values[choice-1])
    elif choice == len(name_of_properties)+1:
      print("По какой характеристике будет группировать? Или на пофиг?")
      while True:
        try:
          for i in range(len(name_of_properties)):
            print(i+1, "Кластеризовать по", name_of_properties[i])
          print(len(name_of_properties)+1, "Посмотреть, что вообще за данные")
          choice_1 = int(input("Введите опцию: "))
          if choice_1 <= len(name_of_properties) and choice_1 >= 1:
            id_array, data, source_array = sort_data(id_array, properties_values[choice_1-1], source_array)
            try:
              sheet_name = input("Результаты будут импортированы в гугл-таблицу. \nИмя листа будет: ")
              worksheet = google_spreedsheet.add_worksheet(title=sheet_name, rows=len(id_array)+1, cols=100)
              prepare_googlesheet(id_array, source_array, data, name_of_properties[choice_1-1], worksheet)
            except gspread.exceptions.APIError:
              worksheet = google_spreedsheet.worksheet(sheet_name)
            break
          elif choice_1 == len(name_of_properties)+1:
            break
          else:
            print("Я не знаю таких цифр")
        except ValueError:
          print("Я не знаю таких цифр")
      while True:
        if choice_1 == len(name_of_properties)+1:
          break
        print("Параметр 'Радиус' показывает, в насколько большом радиусе программа будет собирать числа в кластеры, чем больше параметр -- тем более вероятно слипание близких кластеров\n"
              "Одного единого универсального значения нет!\n"
              "Минимальное в параметрах:  ", np.min(data),
              "\nМаксимальное в параметрах: ", np.max(data))
        while True:
            try:
              epsilon = float(input("Введите радиус: "))
              break
            except ValueError:
              print("Я не знаю таких цифр")
        else:
          print("Я не знаю таких цифр")
        minimum_points = 1
#        print("Параметр 'Минимальнок количество точек' показывает, какое минимальное количество точек в кластере программа будет воспринимать как кластер, а не шум\n",
#              "1. Использовать 1\n",
#              "2. Задать число самому\n")
#        minimum_points = 1
#        choice_2 = input("Введите опцию: ")
#        if choice_2 == '1':
#          minimum_points = 1
#        elif choice_2 == '2':
#          while True:
#            try:
#              minimum_points = int(input("Введите минимальное количество точек (Рекомендуется брать от 3 до 1): "))
#              break
#            except ValueError:
#              print("Я не знаю таких цифр")
#        else:
#          print("Я не знаю таких цифр")
        labels = release_dbscan(data, epsilon, minimum_points)
        packed_clusters = pack_clusters(data, labels, minimum_points)
        import_result_in_googlesheet(packed_clusters, worksheet, epsilon, columns)
        print(f"Данные в гугл-таблице на листе '{sheet_name}' обновлены!\n")
        while True:
          print("Попробовать изменить радиус и/или минамальное количество точек и попробовать снова?\n"
          "1. Да\n"
          "2. Нет\n")
          choice_2 = input("Введите опцию: ")
          if choice_2 == '1':
            break
          elif choice_2 == '2':
            break
          else:
            print("Я не знаю таких цифр")
        if choice_2 == '2':
            break
    elif choice == len(name_of_properties)+2:
      print("Влететь точно в K-means")
      break
    else:
      print("Я не знаю таких цифр")
  except ValueError:
    print("Я не знаю таких цифр")

Есть на просторах интернета одна гугл-таблица, в которой собрано немного про характеристики аэрогелей.
В этой прекрасной гугл-таблице насчитываются характеристики уже 61 аэрогелей
Эта программа может разбивать аэрогели на группы при помощи магии, которая зовётся DBScan
Магия работает по таким характеристикам, как ['Плотность (г/см^3)' 'Уд, поверхность (м^2/г)' 'Размер пор (нм)'
 'Пористость (%)']
1 Посмотреть, что за данные храняться в Плотность (г/см^3)
2 Посмотреть, что за данные храняться в Уд, поверхность (м^2/г)
3 Посмотреть, что за данные храняться в Размер пор (нм)
4 Посмотреть, что за данные храняться в Пористость (%)
5 Начать КЛАСТЕРИЗАЦИЮ!!!!!
 6 Сделать вид что 'подомам'
Введите опцию: 5
По какой характеристике будет группировать? Или на пофиг?
1 Кластеризовать по Плотность (г/см^3)
2 Кластеризовать по Уд, поверхность (м^2/г)
3 Кластеризовать по Размер пор (нм)
4 Кластеризовать по Пористость (%)
5 Посмотреть, что вообще за данные
Введите опцию: 1
Результаты будут импортирова

In [ ]:
print(find_eucledian_distance(properties_values[0][1], properties_values[0][2]))

0.0
